In [1]:
import keras
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices(device_type='GPU')


Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
usaPolyGrid = pkl.load(open("util/usaPolyGrid.pkl",'rb'))
base_path = 'Data/'
grids = np.array(list(usaPolyGrid.keys()),np.int32)

## Format Data

In [3]:

# ds_train = tf.keras.preprocessing.image_dataset_from_directory(
#     'Data/',
#     labels='inferred',
#     label_mode = 'categorical',
#     color_mode='grayscale',
#     batch_size=64,
#     image_size=(1000, 1000),
#     shuffle=True,
#     seed=123,
#     validation_split=.15,
#     subset="training"
# )
#
# ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
#     'Data/',
#     labels='inferred',
#     label_mode = 'categorical',
#     color_mode='grayscale',
#     batch_size=64,
#     image_size=(1000, 1000),
#     shuffle=True,
#     seed=123,
#     validation_split=.15,
#     subset="validation"
# )
batch_size = 20
img_height = 200
img_width = 200

train_ds = tf.keras.utils.image_dataset_from_directory(
    'Simplified_Data',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
    'Simplified_Data',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)



Found 83708 files belonging to 243 classes.
Using 66967 files for training.
Found 83708 files belonging to 243 classes.
Using 16741 files for validation.


In [ ]:
def augment(x,y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return  image,y
train_ds = train_ds.map(augment)

labels = []
for i in range(243):
    labels.append(i)

class_names = val_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")




In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.Rescaling(1./255)


In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))



In [ ]:
num_classes = len(class_names)

model = Sequential([
    layers.Conv2D(32,(5,5), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((5,5)),
    layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
    layers.MaxPooling2D((5,5)),
    layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])
# layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
# layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
# layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
# layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
# layers.Flatten(),
# layers.Dense(1024),
# layers.Dense(num_classes)
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()


In [ ]:
epochs=10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


In [4]:
loaded_model = tf.keras.models.load_model('shane_model')
loaded_model.predict(val_ds)

TypeError: __init__() got an unexpected keyword argument 'ignore_class'

In [ ]:
# import matplotlib.pyplot as plt
#
# plt.figure(figsize=(10, 10))
# for images, labels in ds_train.take(1):
#     for i in range(243):
#         ax = plt.subplot(3,81, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")
#

In [ ]:
# from tensorflow.keras import layers
# from tensorflow import keras
#
# model = keras.Sequential([
#     layers.Input((1000, 1000, 1)),
#     layers.Conv2D(16,3,padding='same'),
#     layers.Conv2D(32,3,padding='same'),
#     layers.MaxPool2D(),
#     layers.Flatten(),
#     layers.Dense(10),
# ])

In [ ]:
# model.compile(optimizer=keras.optimizers.Adam(),
#               loss=[
#                   keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               ],
#               metrics=['accuracy'],
#             )
# model.fit(ds_train, epochs=10, verbose=2)

In [ ]:
# dir = 'locations_train.csv'
# df = pd.read_csv(dir)
#
# file_paths = df['path'].values
# labels = df['grid'].values
# training_data =tf.data.Dataset.from_tensor_slices((file_paths, labels))
# def read_image(image_file, label):
#     image = tf.io.read_file(image_file)
#     image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
#     return image, label
#
# training_data = training_data.map(read_image).batch(64)

In [ ]:
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
# model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))
# model.add(tf.keras.layers.Flatten())
#
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dense(10, activation='softmax'))
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(training_data, epochs=10, verbose=2)